In [2]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer

""" Preparação dos dados
### Mesmo processo feito na parte 1.
"""

url = "https://raw.githubusercontent.com/alura-cursos/alura-clustering-validation/base-de-dados/CC%20GENERAL.csv"

df = pd.read_csv(url)
df.drop(columns=["CUST_ID", "TENURE"], inplace=True)
df = df.fillna(df.median())

valores = Normalizer().fit_transform(df.values)

agrupador = KMeans(n_clusters = 5, n_init = 10, max_iter = 300)
agrupador.fit(valores)

labels = agrupador.labels_
centroides = agrupador.cluster_centers_

df["CLUSTER"] = labels

atributos_selecionados = []

for i in range(len(centroides[0])):
    ## Variancia entre o atributo i em todos os clusteres ([:,i]) 
    variancia = centroides[:, i].var() ## .var() -> variância
    
    if variancia > 0.01:
        atributos_selecionados.append(df.columns.values[i])

atributos_selecionados.append("CLUSTER")

df_selecionado = df[atributos_selecionados].groupby('CLUSTER') ## df dos atributos selecionados agrupado por cluster

qtd_clientes_cluster = df_selecionado.size()

df_selecionado_media = df_selecionado.mean()
df_selecionado_media['QTD_CLIENTES'] = qtd_clientes_cluster
df_selecionado_media

,BALANCE,PURCHASES,CASH_ADVANCE,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,QTD_CLIENTES
CLUSTER,,,,,,,
0,1794.024195,475.494823,3270.246792,3976.372399,4709.556601,734.234709,1072
1,444.212219,629.173858,141.655729,5134.705973,814.396458,228.609769,3276
2,1987.146873,858.326286,420.107194,2226.699029,1338.974045,6032.349113,412
3,3037.962543,385.248630,1636.917210,4495.771989,968.890376,1022.143477,2649
4,1140.395363,3266.466710,182.900631,4096.794388,3034.900772,540.496070,1541


In [5]:
df.groupby("CLUSTER")["PRC_FULL_PAYMENT"].describe() ## % de pagamentos total da fatura

,count,mean,std,min,25%,50%,75%,max
CLUSTER,,,,,,,,
0,1072.0,0.117384,0.186853,0.0,0.0,0.083333,0.142857,1.0
1,3276.0,0.246293,0.346454,0.0,0.0,0.000000,0.444444,1.0
2,412.0,0.019271,0.090795,0.0,0.0,0.000000,0.000000,1.0
3,2649.0,0.001799,0.023260,0.0,0.0,0.000000,0.000000,0.6
4,1541.0,0.279266,0.367477,0.0,0.0,0.083333,0.545455,1.0


Cluster 0:

    - Maior valor em saques;
    - Piores pagadores.

    Ou seja, são clientes que gastam muito com saque.

Cluster 1:

    - Clientes com os menores valores em gastos;
    - Menor valor em saques;
    - Maior limite;
    - Maior quantidade de clientes.

    Ou seja, são clientes que gastam pouco.

Cluster 2:

     - Menor limite;
     - Menor quantidade de clientes.

Cluster 3:

    - Clientes com os maiores valores em gastos;
    - Menor valor em compras;
    - Valor considerável em saques;
    - Piores pagadores (vide PRC_FULL_PAYMENT acima).

    Ou seja, são clientes que mais gastam, com foco em saque.

Cluster 4:

    - Maior valor em compras com grande diferença.

    Ou seja, são clientes que gastam muito com compras.

Um exemplo de aplicação seria:

    - OBJETIVO: Aumentar o lucro da empresa.

Dessa forma, podemos pegar o grupo de clientes com menores gastos e maior limites, por exemplo. Seria o Cluster 1, no qual tem a maior quantidade de clientes e o menor gasto.

Podemos sugerir um programa de pontos, fomentando os clientes a gastar no cartão. Com isso, dividimos os clientes do CLuster 1 em dois grupos, o grupo controle e o grupo teste. Aplicamos o programa de pontos no grupo teste e após um tempo comparamos com o grupo controle. Se o grupo teste gasto mais que o controle, nossa sugestão surtiu efeito, caso contrário, a sugestão não funcionou.